### Прошу прощения, но не смогла выполнить задание в срок. 
Споткнулась буквально сразу же - не могу найти размеченные данные на русском языке для Unigram/Bigram Tagger аналогичные brown (как прикрутить сюда данные из corus я не очень поняла), если брать неразмеченные, не знаю как правильно оценить качество.

Я очень стараюсь победить эту проблему и выполнить задание, буду очень благодарна если датите мне еще несколько дней на это.

## POS-tagger и NER

#### Задание 1. Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы сделать выводы


In [ ]:
# !pip install corus

In [ ]:
# conda install -c anaconda wget

In [ ]:
# conda install -c conda-forge spacy

In [1]:
# Импорт библиотек

import corus
import pandas as pd

import nltk
nltk.download()
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

import warnings
warnings.filterwarnings("ignore")

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [54]:
# data = []

In [56]:
data_tagged = tagging(data)
data_tagged 

In [52]:
# def tagging(sent):
#     sent = nltk.word_tokenize(sent)
#     sent = nltk.pos_tag(sent)
#     return sent

# data_tagged = tagging(data)
# data_tagged 

In [ ]:
# Разбивка данных на тренировочную и тестовые выборки

# train_data = []
# test_data = []

In [49]:
# UnigramTagger

# unigram_tagger = UnigramTagger(train_data)
# display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

In [50]:
# BigramTagger

# bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
# display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

In [ ]:
# TrigramTagger

# trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
# display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

In [ ]:
# Комбинация тэггеров

# def backoff_tagger(train_sents, tagger_classes, backoff=None):
#     for cls in tagger_classes:
#         backoff = cls(train_sents, backoff=backoff)
#     return backoff


# backoff = DefaultTagger('NN') 
# tag = backoff_tagger(train_data,  
#                      [UnigramTagger, BigramTagger, TrigramTagger],  
#                      backoff = backoff) 
  
# tag.evaluate(test_data) 

#### Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
- проверить NER из nltk/spacy/deeppavlov
- написать свой нер попробовать разные подходы
-- передаём в сетку токен и его соседей
-- передаём в сетку только токен
- сделать выводы по вашим экспериментам какой из подходов успешнее справляется
